<a href="https://colab.research.google.com/github/BrianKani/GIS-CropForecast/blob/master/data2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap
import ee

In [2]:
ee.Authenticate()
ee.Initialize(project="ee-briankipsang19")

In [3]:
import time
import pandas as pd
import numpy as np



In [5]:
def export_oneimage(img, folder, name, region, scale, crs):
    task = ee.batch.Export.image(img, name, {
        'driveFolder': folder,
        'driveFileNamePrefix': name,
        'region': region,
        'scale': scale,
        'crs': crs
    })
    task.start()
    while task.status()['state'] == 'RUNNING':
        print('Running...')
        # Perhaps task.cancel() at some point.
        time.sleep(10)
    print('Done.', task.status())

# Read location data from CSV
locations = pd.read_csv('/content/locations.csv')

# Define the bounding box coordinates for Uasin Gishu County
uasin_gishu_bbox = [[34.6898, 0.3601], [35.8078, 0.3601], [35.8078, 0.9558], [34.6898, 0.9558]]

# Transforms an Image Collection with 1 band per Image into a single Image with items as bands
def appendBand(current, previous):
    # Rename the band
    previous = ee.Image(previous)
    current = current.select([0, 1, 2, 3, 4, 5, 6])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous, None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum

# Load the image collection and filter by location and date
imgcoll = ee.ImageCollection('MODIS/MOD09A1') \
    .filterBounds(ee.Geometry.Polygon(uasin_gishu_bbox)) \
    .filterDate('2016-12-31', '2022-12-31')
img = imgcoll.iterate(appendBand)
img = ee.Image(img)

img_0 = ee.Image(ee.Number(0))
img_5000 = ee.Image(ee.Number(5000))

img = img.min(img_5000)
img = img.max(img_0)

# Iterate over locations and export images
for row in locations.values:
    loc1, loc2, lat, lon = row[0], row[1], row[2], row[3]
    fname = '{}_{}'.format(int(loc1), int(loc2))

    offset = 0.11
    scale = 500
    crs = 'EPSG:4326'

    # Define the region of interest as the bounding box of Uasin Gishu County
    region = str(uasin_gishu_bbox)

    while True:
        try:
            export_oneimage(img, 'Data_county', fname, region, scale, crs)
        except Exception as e:
            print('Error:', e)
            print('Retry...')
            time.sleep(10)
            continue
        break

Done. {'state': 'READY', 'description': '1_2', 'creation_timestamp_ms': 1707459643922, 'update_timestamp_ms': 1707459643922, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '2SJLZIS44SUP42TOP6NCHCYP', 'name': 'projects/ee-briankipsang19/operations/2SJLZIS44SUP42TOP6NCHCYP'}
Done. {'state': 'READY', 'description': '1_3', 'creation_timestamp_ms': 1707459644607, 'update_timestamp_ms': 1707459644607, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'DMIKCEXLH23JWYXJSHXPZ73M', 'name': 'projects/ee-briankipsang19/operations/DMIKCEXLH23JWYXJSHXPZ73M'}
Done. {'state': 'READY', 'description': '1_4', 'creation_timestamp_ms': 1707459645141, 'update_timestamp_ms': 1707459645141, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'AEJB32ORW5LO2ZN7XJX2PJML', 'name': 'projects/ee-briankipsang19/operations/AEJB32ORW5LO2ZN7XJX2PJML'}
Done. {'state': 'READY', 'description': '1_5', 'creation_timestamp_ms': 1707459645687, 'update_timestamp_ms': 1707459645687, 'start_timesta